In [1]:
import pandas as pd
import numpy as np
import time
import joblib
import collections
from sklearn import preprocessing
from tensorflow.keras.models import model_from_json
from tensorflow.keras import layers
from tensorflow.keras import optimizers

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df_test = pd.read_csv('data/df_test.csv')

In [3]:
df_test.head()

,TenantId,CreatedDate,CreateDateOfFirstSubscription,CountryCode,AllupSeats,EXOSubscriptionsCount,OD4BSubscriptionsCount,SfBSubscriptionsCount,TeamsSubscriptionsCount,PaidCount,ProjectSubscriptionsCount,SPOSubscriptionsCount,ActivatedSubscriptionTotalCount,VisioSubscriptionsCount,TrialSubscriptionsCount,NonTrialSubscriptionsCount,Languange,DataCenterInstance,DataCenterModel,HasEXO,HasSPO,HasOD4B,HasSfB,HasYammer,HasTeams,HasTeamsFreemium,HasKaizala,HasProPlus,HasAADP,HasAIP,HasAATP,HasIntune,HasMCAS,HasWDATP,HasAudioConference,HasPhoneSystem,HasEdiscovery,HasCompliance,HasThreatIntelligence,HasCustomerLockbox,HasOATP,HasAADPP2,HasAIPP2,HasWindows,HasO365CAS,HasCASDiscovery,HasPAM,HasPowerBI,HasPowerBIPremium,HasPowerBIPro,HasVisio,HasProject,HasNonTrial,HasSubscription_AllCounted,WithoutTenantAdmin,OrgNameEnteredInCommerce,HasPartnerTenants,SignupLocationInfo_Country,SignupLocationInfo_CountryCode,SignupLocationInfo_Region,Subscriptions_HasPaidSeats,TopParents_AreaName,TopParents_BigAreaName,TopParents_CountryCode,TopParents_Industry,TopParents_RegionName,TopParents_SegmentGroup,TopParents_SubRegionName,TopParents_VerticalName,EXOEnabledUsers,SPOEnabledUsers,OD4BEnabledUsers,SFBEnabledUsers,TeamEnabledUsers,YammerEnabledUsers,PPDEnabledUsers,KaizalaEnabledUsers,AADPEnabledUsers,AIPEnabledUsers,AATPEnabledUsers,IntuneEnabledUsers,MCASEnabledUsers,WDATPEnabledUsers,AudioConferenceEnabledUsers,PhoneSystemEnabledUsers,EdiscoveryEnabledUsers,ComplianceEnabledUsers,ThreatIntelligenceEnabledUsers,CustomerLockboxEnabledUsers,OATPEnabledUsers,AADPP2EnabledUsers,AIPP2EnabledUsers,WindowsEnabledUsers,O365CASEnabledUsers,CASDiscoveryEnabledUsers,PAMEnabledUsers,O365EnabledUsers,EMSEnabledUsers,M365EnabledUsers,O365E5EnabledUsers,EMSE5EnabledUsers,M365E5EnabledUsers,TotalUsers,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidYammerSeats,PaidTeamsSeats,PaidSFBSeats,PaidKaizalaSeats,PaidProplusSeats,PaidAADPSeats,PaidAIPSeats,PaidAATPSeats,PaidIntuneSeats,PaidMCASSeats,PaidWDATPSeats,PaidAudioConferenceSeats,PaidPhoneSystemSeats,PaidOATPSeats,PaidAADPP2Seats,PaidAIPP2Seats,PaidWindowsSeats,PaidO365CASSeats,PaidCASDiscoverySeats,PaidPAMSeats,PaidPowerBISeats,PaidPowerBIPremiumSeats,PaidPowerBIProSeats,PaidEMSSeats,PaidM365Seats,PaidOfficeSeats,HasUsGovCloudOffer,FirstPaidEXOStartDate,FirstPaidSPOStartDate,FirstPaidOD4BStartDate,FirstPaidSfBStartDate,FirstPaidYammerStartDate,FirstPaidTeamsStartDate,FirstPaidProPlusStartDate,FirstPaidAADPStartDate,FirstPaidAIPStartDate,FirstPaidAATPStartDate,FirstPaidIntuneStartDate,FirstPaidMCASStartDate,FirstPaidO365E5SkuStartDate,FirstPaidM365E5SkuStartDate,FirstPaidEMSE5SkuStartDate,Has1YearFreeDomain,AU_exchange_04,AU_sharepoint_04,AU_skype_04,AU_teams_04,AU_od4b_04,AU_onenote_04,AU_word_04,AU_excel_04,AU_powerpoint_04,AU_outlook_04,AU_oatp_04,AU_odsp_04,AU_eslt_04,AU_officeclient_04,AU_exchange_05,AU_sharepoint_05,AU_skype_05,AU_teams_05,AU_od4b_05,AU_onenote_05,AU_word_05,AU_excel_05,AU_powerpoint_05,AU_outlook_05,AU_oatp_05,AU_odsp_05,AU_eslt_05,AU_officeclient_05,AU_exchange,AU_sharepoint,AU_skype,AU_teams,AU_od4b,AU_onenote,AU_word,AU_excel,AU_powerpoint,AU_outlook,AU_oatp,AU_odsp,AU_eslt,AU_officeclient,AR_exchange_04,AR_sharepoint_04,AR_skype_04,AR_teams_04,AR_od4b_04,AR_onenote_04,AR_word_04,AR_excel_04,AR_powerpoint_04,AR_outlook_04,AR_eslt_04,AR_officeclient_04,AR_exchange_05,AR_sharepoint_05,AR_skype_05,AR_teams_05,AR_od4b_05,AR_onenote_05,AR_word_05,AR_excel_05,AR_powerpoint_05,AR_outlook_05,AR_eslt_05,AR_officeclient_05,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06
0,946A2663-3AD0-46B8-90C0-DE373075DCCE,2/5/2014 5:31:19 AM,11/14/2018 12:00:00 AM,BE,5,1,0,1,0,5,0,0,2,0,0,2,en,Global|GoLocal,PublicCloud|GoLocal,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [4]:
df_test.shape

(478934, 227)

In [11]:
Xtest = np.load('data/Xtest.npy')
ytest = np.load('data/ytest.npy')

# ytest.shape

In [6]:
outputs_name = [
     'AR_exchange_06',
 'AR_sharepoint_06',
 'AR_skype_06',
 'AR_teams_06',
 'AR_od4b_06',
 'AR_onenote_06',
 'AR_word_06',
 'AR_excel_06',
 'AR_powerpoint_06',
 'AR_outlook_06',
 'AR_eslt_06',
 'AR_officeclient_06'
]

In [7]:
display_cols = ['TenantId', 'CountryCode', 'TopParents_Industry','AllupSeats', 
                'PaidCount','PaidEXOSeats', 'PaidSPOSeats', 'PaidOD4BSeats',
                'PaidTeamsSeats', 'PaidSFBSeats', 'PaidOfficeSeats'
               ] + outputs_name

In [8]:
df_display = df_test.loc[:, display_cols]

In [9]:
df_display.head()

,TenantId,CountryCode,TopParents_Industry,AllupSeats,PaidCount,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidTeamsSeats,PaidSFBSeats,PaidOfficeSeats,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06
0,946A2663-3AD0-46B8-90C0-DE373075DCCE,BE,0,5,5,4,0,0,0,1,0.0,0.550000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.440000,0.080000
1,61BEFEAC-AAD0-4D27-80DB-BEA99BAA3276,BE,Consumer Goods,99,99,98,98,99,98,98,0.0,0.756122,0.000000,0.002551,0.000510,0.117677,0.134848,0.304545,0.335354,0.211111,0.566667,0.753535,0.593434
2,96847494-5BF3-483A-954E-0205FCC59791,IL,Other - Unsegmented,190,188,181,176,182,175,175,0.0,0.786188,0.041477,0.000000,0.026571,0.023352,0.005851,0.175266,0.160638,0.040691,0.335372,0.761170,0.353457
3,C80F14D1-98F4-4E83-94E7-376CC5B34D63,US,Discrete Manufacturing,12,12,12,12,12,12,12,0.0,0.316667,0.000000,0.000000,0.000000,0.012500,0.000000,0.120833,0.108333,0.000000,0.325000,0.316667,0.341667
4,7E8FB864-A888-49CF-81B6-C072D37162DA,US,Smart Spaces,27,2,2,2,2,2,2,0.0,1.500000,1.050000,0.000000,0.000000,0.350000,0.000000,0.550000,0.175000,0.000000,1.500000,1.500000,1.500000


In [10]:
df_display.shape

(478934, 23)

In [12]:
# load json and create model
model_n = 17
model_path = 'results/NNmodel_{}_with_1.0data.json'.format(model_n)
print(model_path)
weights_path = 'results/NNmodel_{}_with_1.0data_weights.hdf5'.format(model_n)

json_file = open(model_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(weights_path)
print("Loaded model from disk")

# evaluate loaded model 
loaded_model.compile(loss='mean_squared_error', optimizer='adam',
                     metrics=['mean_squared_error'])

W0827 22:53:57.046210 140426779870976 deprecation.py:506] From /data/home/t-chepan/env/newlab/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0827 22:53:57.047366 140426779870976 deprecation.py:506] From /data/home/t-chepan/env/newlab/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0827 22:53:57.048409 140426779870976 deprecation.py:506] From /data/home/t-chepan/env/newlab/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ze

results/NNmodel_17_with_1.0data.json
Loaded model from disk


In [13]:
test_pred = loaded_model.predict(Xtest)

In [14]:
for i in range(len(test_pred[1])):
    col_name = outputs_name[i] +'_delta'
    df_display[col_name] = test_pred[:,i] - ytest[:,i]

## use some criteria to filter the df_display

1. filter out the tenants without "TopParent_Industry" info
2. "PaidTeamsSeats" should equal to "PaidSFBSeats". filter out the tenant if these two don't match

In [16]:
df_display = df_display.loc[(df_display['TopParents_Industry'] != '0') &
                            (df_display['PaidSFBSeats'] == df_display['PaidTeamsSeats'])]

In [17]:
df_display.shape

(218943, 35)

In [18]:
df_display.head()

,TenantId,CountryCode,TopParents_Industry,AllupSeats,PaidCount,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidTeamsSeats,PaidSFBSeats,PaidOfficeSeats,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06,AR_exchange_06_delta,AR_sharepoint_06_delta,AR_skype_06_delta,AR_teams_06_delta,AR_od4b_06_delta,AR_onenote_06_delta,AR_word_06_delta,AR_excel_06_delta,AR_powerpoint_06_delta,AR_outlook_06_delta,AR_eslt_06_delta,AR_officeclient_06_delta
1,61BEFEAC-AAD0-4D27-80DB-BEA99BAA3276,BE,Consumer Goods,99,99,98,98,99,98,98,0.0,0.756122,0.000000,0.002551,0.000510,0.117677,0.134848,0.304545,0.335354,0.211111,0.566667,0.753535,0.593434,-0.029681,0.111008,0.026397,0.070267,-0.001255,-0.115424,-0.041749,-0.034330,-0.143518,0.043503,-0.021852,0.057970
2,96847494-5BF3-483A-954E-0205FCC59791,IL,Other - Unsegmented,190,188,181,176,182,175,175,0.0,0.786188,0.041477,0.000000,0.026571,0.023352,0.005851,0.175266,0.160638,0.040691,0.335372,0.761170,0.353457,0.061550,0.046782,0.025641,0.036603,0.075187,0.011901,0.045637,0.091641,0.019565,0.224701,0.088784,0.224936
3,C80F14D1-98F4-4E83-94E7-376CC5B34D63,US,Discrete Manufacturing,12,12,12,12,12,12,12,0.0,0.316667,0.000000,0.000000,0.000000,0.012500,0.000000,0.120833,0.108333,0.000000,0.325000,0.316667,0.341667,-0.028725,0.021764,-0.000091,0.018757,0.031787,0.009714,-0.010614,0.014761,0.022720,-0.112116,-0.024838,-0.081048
4,7E8FB864-A888-49CF-81B6-C072D37162DA,US,Smart Spaces,27,2,2,2,2,2,2,0.0,1.500000,1.050000,0.000000,0.000000,0.350000,0.000000,0.550000,0.175000,0.000000,1.500000,1.500000,1.500000,0.038967,-0.339330,0.060534,0.012454,-0.091890,0.063188,-0.121814,0.216621,0.077620,-0.126477,0.129995,-0.076156
5,FBDE669A-8B4E-4E6C-A078-C1A71E185C52,IT,Chemicals & Agrochemicals,160,50,4,5,50,4,4,0.0,0.000000,0.300000,0.000000,0.000000,0.042000,0.000000,0.394000,0.544000,0.084000,0.578000,0.071000,0.608000,0.251292,-0.256767,-0.003927,0.024756,-0.005063,0.010471,0.237101,0.132118,-0.012507,0.370985,0.230936,0.411640


## Mark each workload as 1/0/-1 (Hackathon)

In [19]:
delta_cols = df_display.columns[-12:]

paidseats = ['PaidEXOSeats', 'PaidSPOSeats', 'PaidSFBSeats','PaidTeamsSeats',
             'PaidOD4BSeats', 'PaidOfficeSeats', 'PaidOfficeSeats','PaidOfficeSeats',
             'PaidOfficeSeats','PaidOfficeSeats','PaidCount', 'PaidOfficeSeats']

targets = list(zip(paidseats, delta_cols, outputs_name))

In [20]:
threshold = 0.05

for ps, delta, acr in targets:
    def f(row):
        if row[ps] <= 0:
            val = -1
        else:
            if row[delta] > threshold and row[acr] < 1:
                val = 1
            else:
                val = 0
        return val
    
    vals = []
    for i, row in df_display.iterrows():
        vals.append(f(row))
        
    df_display['Rec_{}'.format(acr)] = vals


In [22]:
df_display.head()

,TenantId,CountryCode,TopParents_Industry,AllupSeats,PaidCount,PaidEXOSeats,PaidSPOSeats,PaidOD4BSeats,PaidTeamsSeats,PaidSFBSeats,PaidOfficeSeats,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06,AR_exchange_06_delta,AR_sharepoint_06_delta,AR_skype_06_delta,AR_teams_06_delta,AR_od4b_06_delta,AR_onenote_06_delta,AR_word_06_delta,AR_excel_06_delta,AR_powerpoint_06_delta,AR_outlook_06_delta,AR_eslt_06_delta,AR_officeclient_06_delta,Rec_AR_exchange_06,Rec_AR_sharepoint_06,Rec_AR_skype_06,Rec_AR_teams_06,Rec_AR_od4b_06,Rec_AR_onenote_06,Rec_AR_word_06,Rec_AR_excel_06,Rec_AR_powerpoint_06,Rec_AR_outlook_06,Rec_AR_eslt_06,Rec_AR_officeclient_06
1,61BEFEAC-AAD0-4D27-80DB-BEA99BAA3276,BE,Consumer Goods,99,99,98,98,99,98,98,0.0,0.756122,0.000000,0.002551,0.000510,0.117677,0.134848,0.304545,0.335354,0.211111,0.566667,0.753535,0.593434,-0.029681,0.111008,0.026397,0.070267,-0.001255,-0.115424,-0.041749,-0.034330,-0.143518,0.043503,-0.021852,0.057970,0,1,0,1,0,-1,-1,-1,-1,-1,0,-1
2,96847494-5BF3-483A-954E-0205FCC59791,IL,Other - Unsegmented,190,188,181,176,182,175,175,0.0,0.786188,0.041477,0.000000,0.026571,0.023352,0.005851,0.175266,0.160638,0.040691,0.335372,0.761170,0.353457,0.061550,0.046782,0.025641,0.036603,0.075187,0.011901,0.045637,0.091641,0.019565,0.224701,0.088784,0.224936,1,0,0,0,1,-1,-1,-1,-1,-1,1,-1
3,C80F14D1-98F4-4E83-94E7-376CC5B34D63,US,Discrete Manufacturing,12,12,12,12,12,12,12,0.0,0.316667,0.000000,0.000000,0.000000,0.012500,0.000000,0.120833,0.108333,0.000000,0.325000,0.316667,0.341667,-0.028725,0.021764,-0.000091,0.018757,0.031787,0.009714,-0.010614,0.014761,0.022720,-0.112116,-0.024838,-0.081048,0,0,0,0,0,-1,-1,-1,-1,-1,0,-1
4,7E8FB864-A888-49CF-81B6-C072D37162DA,US,Smart Spaces,27,2,2,2,2,2,2,0.0,1.500000,1.050000,0.000000,0.000000,0.350000,0.000000,0.550000,0.175000,0.000000,1.500000,1.500000,1.500000,0.038967,-0.339330,0.060534,0.012454,-0.091890,0.063188,-0.121814,0.216621,0.077620,-0.126477,0.129995,-0.076156,0,0,1,0,0,-1,-1,-1,-1,-1,0,-1
5,FBDE669A-8B4E-4E6C-A078-C1A71E185C52,IT,Chemicals & Agrochemicals,160,50,4,5,50,4,4,0.0,0.000000,0.300000,0.000000,0.000000,0.042000,0.000000,0.394000,0.544000,0.084000,0.578000,0.071000,0.608000,0.251292,-0.256767,-0.003927,0.024756,-0.005063,0.010471,0.237101,0.132118,-0.012507,0.370985,0.230936,0.411640,1,0,0,0,0,-1,-1,-1,-1,-1,1,-1


## Display the rank of AR_delta and the valid choice for agent

In [73]:
mask_paidseats = np.zeros((df_display.shape[0],12))
for i, ps in enumerate(paidseats):
    mask_paidseats[:,i] = np.where(df_display[ps]>0, 1, 0 )
    

In [74]:
np.sum(mask_paidseats)

1202246.0

In [75]:
delta_cols

Index(['AR_exchange_06_delta', 'AR_sharepoint_06_delta', 'AR_skype_06_delta',
       'AR_teams_06_delta', 'AR_od4b_06_delta', 'AR_onenote_06_delta',
       'AR_word_06_delta', 'AR_excel_06_delta', 'AR_powerpoint_06_delta',
       'AR_outlook_06_delta', 'AR_eslt_06_delta', 'AR_officeclient_06_delta'],
      dtype='object')

In [76]:
df_delta = df_display.loc[:, delta_cols]

In [77]:
df_delta = df_delta * mask_paidseats

In [78]:
r_dict = dict(df_delta.iloc[0,:])
sorted_r = sorted(r_dict.items(), key=lambda kv: kv[1], reverse=True)
sorted_r[:2]

[('AR_sharepoint_06_delta', 0.11100785434246063),
 ('AR_teams_06_delta', 0.07026733880748554)]

In [81]:
r_dict = dict(df_delta.iloc[0,:])
valid_wl = []
new_r_dict = {}

for k, v in r_dict.items():
    k = k[3:-9]
    if v != 0.0:
        valid_wl.append(k)
    if v > 0.0:
        new_r_dict[k] = v

new_r_dict = collections.OrderedDict(sorted(new_r_dict.items(), key=lambda kv: kv[1], reverse=True))
        
print(list(new_r_dict.keys()))
        
print(valid_wl)

['sharepoint', 'teams', 'skype']
['od4b', 'sharepoint', 'exchange', 'teams', 'eslt', 'skype']


In [82]:
def f(row):
    r_dict = dict(row)
    valid_wl = []
    new_r_dict = {}

    for k, v in r_dict.items():
        k = k[3:-9]
        if v != 0.0:
            valid_wl.append(k)
        if v > 0.0:
            new_r_dict[k] = v

    new_r_dict = collections.OrderedDict(sorted(new_r_dict.items(), key=lambda kv: kv[1], reverse=True))
    
    return valid_wl, list(new_r_dict.keys())

In [83]:
valid_wl = []
rec = []
for i, row in df_delta.iterrows():
    wl, rank = f(row)
    valid_wl.append(wl)
    rec.append(rank)
    
df_delta['valid_choice'] = valid_wl
df_delta['rec'] = rec

    

In [84]:
df_delta.head()

,AR_exchange_06_delta,AR_sharepoint_06_delta,AR_skype_06_delta,AR_teams_06_delta,AR_od4b_06_delta,AR_onenote_06_delta,AR_word_06_delta,AR_excel_06_delta,AR_powerpoint_06_delta,AR_outlook_06_delta,AR_eslt_06_delta,AR_officeclient_06_delta,valid_choice,rec
1,-0.029681,0.111008,0.026397,0.070267,-0.001255,-0.0,-0.0,-0.0,-0.0,0.0,-0.021852,0.0,"[od4b, sharepoint, exchange, teams, eslt, skype]","[sharepoint, teams, skype]"
2,0.061550,0.046782,0.025641,0.036603,0.075187,0.0,0.0,0.0,0.0,0.0,0.088784,0.0,"[od4b, sharepoint, exchange, teams, eslt, skype]","[eslt, od4b, exchange, sharepoint, teams, skype]"
3,-0.028725,0.021764,-0.000091,0.018757,0.031787,0.0,-0.0,0.0,0.0,-0.0,-0.024838,-0.0,"[od4b, sharepoint, exchange, teams, eslt, skype]","[od4b, sharepoint, teams]"
4,0.038967,-0.339330,0.060534,0.012454,-0.091890,0.0,-0.0,0.0,0.0,-0.0,0.129995,-0.0,"[od4b, sharepoint, exchange, teams, eslt, skype]","[eslt, skype, exchange, teams]"
5,0.251292,-0.256767,-0.003927,0.024756,-0.005063,0.0,0.0,0.0,-0.0,0.0,0.230936,0.0,"[od4b, sharepoint, exchange, teams, eslt, skype]","[exchange, eslt, teams]"


In [85]:
df_delta.shape

(218943, 14)

In [86]:
df_display.shape

(218943, 47)

In [1]:
df_display.head(10)

NameError: name 'df_display' is not defined

In [88]:
df_display_full = pd.concat([df_display,
                             df_delta.iloc[:, -2:]], axis=1)

In [89]:
df_display_full.shape

(218943, 49)

In [90]:
df_display_full.to_csv('results/rec_dispaly_full.csv', index=False)

In [91]:
show_cols = ['TenantId', 'CountryCode', 'TopParents_Industry','AllupSeats',
             'PaidCount'] + outputs_name + ['valid_choice', 'rec']

df_display_show = df_display_full.loc[:, show_cols]

In [109]:
df_display_show.head(9)

,TenantId,CountryCode,TopParents_Industry,AllupSeats,PaidCount,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06,valid_choice,rec
1,61BEFEAC-AAD0-4D27-80DB-BEA99BAA3276,BE,Consumer Goods,99,99,0.756122,0.000000,0.002551,0.000510,0.117677,0.134848,0.304545,0.335354,0.211111,0.566667,0.753535,0.593434,"[od4b, sharepoint, exchange, teams, eslt, skype]","[sharepoint, teams, skype]"
2,96847494-5BF3-483A-954E-0205FCC59791,IL,Other - Unsegmented,190,188,0.786188,0.041477,0.000000,0.026571,0.023352,0.005851,0.175266,0.160638,0.040691,0.335372,0.761170,0.353457,"[od4b, sharepoint, exchange, teams, eslt, skype]","[eslt, od4b, exchange, sharepoint, teams, skype]"
3,C80F14D1-98F4-4E83-94E7-376CC5B34D63,US,Discrete Manufacturing,12,12,0.316667,0.000000,0.000000,0.000000,0.012500,0.000000,0.120833,0.108333,0.000000,0.325000,0.316667,0.341667,"[od4b, sharepoint, exchange, teams, eslt, skype]","[od4b, sharepoint, teams]"
5,FBDE669A-8B4E-4E6C-A078-C1A71E185C52,IT,Chemicals & Agrochemicals,160,50,0.000000,0.300000,0.000000,0.000000,0.042000,0.000000,0.394000,0.544000,0.084000,0.578000,0.071000,0.608000,"[od4b, sharepoint, exchange, teams, eslt, skype]","[exchange, eslt, teams]"
11,93F7AC5D-F331-4033-98D9-AFE7CBB6CE43,CA,Travel & Transportation,69,68,0.847674,0.027273,0.000000,0.012500,0.015909,0.030147,0.108824,0.108824,0.002206,0.158824,0.541176,0.163235,"[od4b, sharepoint, exchange, teams, eslt, skype]","[eslt, exchange, teams]"
13,66EBF719-001C-400D-8AEF-AEC9FBCD0AD3,AU,Smart Spaces,116,66,0.866949,0.760577,0.248039,0.950000,0.323276,0.094697,0.368939,0.421212,0.161364,0.743939,0.983333,0.765152,"[od4b, sharepoint, exchange, teams, eslt, skype]",[]
14,014BA161-8B87-47FF-8F30-9AA6E5635CF7,GB,Smart Spaces,28,28,0.860714,0.000000,0.000000,0.021154,0.011538,0.016071,0.300000,0.257143,0.058929,0.444643,0.860714,0.450000,"[od4b, sharepoint, exchange, teams, eslt, skype]","[sharepoint, od4b, exchange, eslt, teams, skype]"
15,08E290CC-D76D-4BEF-A32E-14D7FE9AD38E,NO,Consumer Goods,17,17,0.744118,0.000000,0.000000,0.000000,0.004545,0.000000,0.158824,0.202941,0.105882,0.411765,0.744118,0.414706,"[od4b, sharepoint, exchange, teams, eslt, skype]","[od4b, sharepoint, teams, skype]"
17,C78822F4-7348-4759-9776-5D111BF82948,DK,Media & Entertainment,3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.050000,0.000000,0.000000,0.000000,0.050000,"[od4b, eslt]",[od4b]


In [119]:
fake_df = df_display_show.head(9)
fake_df.shape

(9, 19)

In [120]:
fake_df.loc[fake_df.astype(str)['rec'] != '[]']

,TenantId,CountryCode,TopParents_Industry,AllupSeats,PaidCount,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06,valid_choice,rec
1,61BEFEAC-AAD0-4D27-80DB-BEA99BAA3276,BE,Consumer Goods,99,99,0.756122,0.000000,0.002551,0.000510,0.117677,0.134848,0.304545,0.335354,0.211111,0.566667,0.753535,0.593434,"[od4b, sharepoint, exchange, teams, eslt, skype]","[sharepoint, teams, skype]"
2,96847494-5BF3-483A-954E-0205FCC59791,IL,Other - Unsegmented,190,188,0.786188,0.041477,0.000000,0.026571,0.023352,0.005851,0.175266,0.160638,0.040691,0.335372,0.761170,0.353457,"[od4b, sharepoint, exchange, teams, eslt, skype]","[eslt, od4b, exchange, sharepoint, teams, skype]"
3,C80F14D1-98F4-4E83-94E7-376CC5B34D63,US,Discrete Manufacturing,12,12,0.316667,0.000000,0.000000,0.000000,0.012500,0.000000,0.120833,0.108333,0.000000,0.325000,0.316667,0.341667,"[od4b, sharepoint, exchange, teams, eslt, skype]","[od4b, sharepoint, teams]"
5,FBDE669A-8B4E-4E6C-A078-C1A71E185C52,IT,Chemicals & Agrochemicals,160,50,0.000000,0.300000,0.000000,0.000000,0.042000,0.000000,0.394000,0.544000,0.084000,0.578000,0.071000,0.608000,"[od4b, sharepoint, exchange, teams, eslt, skype]","[exchange, eslt, teams]"
11,93F7AC5D-F331-4033-98D9-AFE7CBB6CE43,CA,Travel & Transportation,69,68,0.847674,0.027273,0.000000,0.012500,0.015909,0.030147,0.108824,0.108824,0.002206,0.158824,0.541176,0.163235,"[od4b, sharepoint, exchange, teams, eslt, skype]","[eslt, exchange, teams]"
14,014BA161-8B87-47FF-8F30-9AA6E5635CF7,GB,Smart Spaces,28,28,0.860714,0.000000,0.000000,0.021154,0.011538,0.016071,0.300000,0.257143,0.058929,0.444643,0.860714,0.450000,"[od4b, sharepoint, exchange, teams, eslt, skype]","[sharepoint, od4b, exchange, eslt, teams, skype]"
15,08E290CC-D76D-4BEF-A32E-14D7FE9AD38E,NO,Consumer Goods,17,17,0.744118,0.000000,0.000000,0.000000,0.004545,0.000000,0.158824,0.202941,0.105882,0.411765,0.744118,0.414706,"[od4b, sharepoint, exchange, teams, eslt, skype]","[od4b, sharepoint, teams, skype]"
17,C78822F4-7348-4759-9776-5D111BF82948,DK,Media & Entertainment,3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.050000,0.000000,0.000000,0.000000,0.050000,"[od4b, eslt]",[od4b]


In [94]:
pd.options.display.max_colwidth = 100


In [121]:
df_display_show = df_display_show.loc[(df_display_show.AR_exchange_06 <= 1.0) &
                                      (df_display_show.AR_sharepoint_06 <= 1.0) &
                                      (df_display_show.AR_skype_06 <= 1.0) &
                                      (df_display_show.AR_teams_06 <= 1.0) &
                                      (df_display_show.AR_od4b_06 <= 1.0) &
                                      (df_display_show.AR_eslt_06 <= 1.0) & 
                                      (df_display_show.AR_onenote_06 <= 1.0) &
                                      (df_display_show.AR_word_06 <= 1.0) &
                                      (df_display_show.AR_excel_06 <= 1.0) & 
                                      (df_display_show.AR_powerpoint_06 <= 1.0) &
                                      (df_display_show.AR_outlook_06 <= 1.0) &
                                      (df_display_show.AR_officeclient_06 <= 1.0) &
                                      (df_display_show.astype(str)['rec'] != '[]')]

In [122]:
df_display_show.shape

(148491, 19)

In [123]:
df_display_show.columns = ['TenantId', 'CountryCode', 'TopParents_Industry',
                           'AllupSeats','PaidCount', 'exchange_usage','sharepoint_usage',
                           'skype_usage','teams_usage','od4b_usage','onenote_usage',
                           'word_usage','excel_usage','powerpoint_usage','outlook_usage',
                           'eslt_usage','officeclient_usage', 'Valid_Choice','Recommender from Model']

In [125]:
df_display_show.head(10)

,TenantId,CountryCode,TopParents_Industry,AllupSeats,PaidCount,exchange_usage,sharepoint_usage,skype_usage,teams_usage,od4b_usage,onenote_usage,word_usage,excel_usage,powerpoint_usage,outlook_usage,eslt_usage,officeclient_usage,Valid_Choice,Recommender from Model
1,61BEFEAC-AAD0-4D27-80DB-BEA99BAA3276,BE,Consumer Goods,99,99,0.756122,0.000000,0.002551,0.000510,0.117677,0.134848,0.304545,0.335354,0.211111,0.566667,0.753535,0.593434,"[od4b, sharepoint, exchange, teams, eslt, skype]","[sharepoint, teams, skype]"
2,96847494-5BF3-483A-954E-0205FCC59791,IL,Other - Unsegmented,190,188,0.786188,0.041477,0.000000,0.026571,0.023352,0.005851,0.175266,0.160638,0.040691,0.335372,0.761170,0.353457,"[od4b, sharepoint, exchange, teams, eslt, skype]","[eslt, od4b, exchange, sharepoint, teams, skype]"
3,C80F14D1-98F4-4E83-94E7-376CC5B34D63,US,Discrete Manufacturing,12,12,0.316667,0.000000,0.000000,0.000000,0.012500,0.000000,0.120833,0.108333,0.000000,0.325000,0.316667,0.341667,"[od4b, sharepoint, exchange, teams, eslt, skype]","[od4b, sharepoint, teams]"
5,FBDE669A-8B4E-4E6C-A078-C1A71E185C52,IT,Chemicals & Agrochemicals,160,50,0.000000,0.300000,0.000000,0.000000,0.042000,0.000000,0.394000,0.544000,0.084000,0.578000,0.071000,0.608000,"[od4b, sharepoint, exchange, teams, eslt, skype]","[exchange, eslt, teams]"
11,93F7AC5D-F331-4033-98D9-AFE7CBB6CE43,CA,Travel & Transportation,69,68,0.847674,0.027273,0.000000,0.012500,0.015909,0.030147,0.108824,0.108824,0.002206,0.158824,0.541176,0.163235,"[od4b, sharepoint, exchange, teams, eslt, skype]","[eslt, exchange, teams]"
14,014BA161-8B87-47FF-8F30-9AA6E5635CF7,GB,Smart Spaces,28,28,0.860714,0.000000,0.000000,0.021154,0.011538,0.016071,0.300000,0.257143,0.058929,0.444643,0.860714,0.450000,"[od4b, sharepoint, exchange, teams, eslt, skype]","[sharepoint, od4b, exchange, eslt, teams, skype]"
15,08E290CC-D76D-4BEF-A32E-14D7FE9AD38E,NO,Consumer Goods,17,17,0.744118,0.000000,0.000000,0.000000,0.004545,0.000000,0.158824,0.202941,0.105882,0.411765,0.744118,0.414706,"[od4b, sharepoint, exchange, teams, eslt, skype]","[od4b, sharepoint, teams, skype]"
17,C78822F4-7348-4759-9776-5D111BF82948,DK,Media & Entertainment,3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.050000,0.000000,0.000000,0.000000,0.050000,"[od4b, eslt]",[od4b]
18,C00C9442-9BB5-4D3B-86E7-BCB72E7FF9CF,BR,Travel & Transportation,114,114,0.918692,0.000000,0.000000,0.000000,0.000000,0.000439,0.133333,0.196053,0.008772,0.249561,0.862281,0.253509,"[od4b, exchange, eslt]",[od4b]
19,CAD4C6D0-7413-4405-A069-0CC7D7DA9F2B,CA,Nonprofit,171,151,0.654305,0.037500,0.000000,0.161429,0.027857,0.004305,0.124503,0.092384,0.017219,0.346026,0.661921,0.358940,"[od4b, sharepoint, exchange, teams, eslt, skype]","[od4b, sharepoint, skype, exchange, eslt]"


In [126]:
df_display_show_save = df_display_show.iloc[:500,:]
df_display_show_save.to_excel('results/rec_display_show_agent.xlsx', index=False)

In [30]:
# df_display = pd.read_csv('results/rec_display.csv')

In [110]:
df_display_rank.to_csv('results/rec_dispaly_rank.csv', index=False)

In [8]:
df_display_s_show = pd.read_csv('results/rec_dispaly_full.csv',
#                                 nrows=100
                               )

In [9]:
df_display_rank_show = pd.read_csv('results/rec_dispaly_rank.csv',
#                                    nrows=100
                                  )

In [14]:
df_display_s_show.shape

(478934, 41)

In [11]:
valid_tenantid = df_display_s_show.loc[(df_display_s_show['TopParents_Industry'] != '0') &
                      (df_display_s_show['Rec_AR_skype_06'] == df_display_s_show['Rec_AR_teams_06']), 'TenantId']

In [20]:
valid_display_df = df_display_s_show.loc[df_display_s_show['TenantId'].isin(valid_tenantid)]

In [22]:
valid_display_df.head()

,TenantId,CountryCode,TopParents_Industry,AllupSeats,PaidCount,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06,AR_exchange_06_delta,AR_sharepoint_06_delta,AR_skype_06_delta,AR_teams_06_delta,AR_od4b_06_delta,AR_onenote_06_delta,AR_word_06_delta,AR_excel_06_delta,AR_powerpoint_06_delta,AR_outlook_06_delta,AR_eslt_06_delta,AR_officeclient_06_delta,Rec_AR_exchange_06,Rec_AR_sharepoint_06,Rec_AR_skype_06,Rec_AR_teams_06,Rec_AR_od4b_06,Rec_AR_onenote_06,Rec_AR_word_06,Rec_AR_excel_06,Rec_AR_powerpoint_06,Rec_AR_outlook_06,Rec_AR_eslt_06,Rec_AR_officeclient_06
2,96847494-5BF3-483A-954E-0205FCC59791,IL,Other - Unsegmented,190,188,0.786188,0.041477,0.000000,0.026571,0.023352,0.005851,0.175266,0.160638,0.040691,0.335372,0.761170,0.353457,0.104305,0.013027,0.017809,0.021968,0.047962,0.008793,-0.016593,0.019705,0.005814,0.116238,0.128416,0.097951,1,0,0,0,0,-1,-1,-1,-1,-1,1,-1
3,C80F14D1-98F4-4E83-94E7-376CC5B34D63,US,Discrete Manufacturing,12,12,0.316667,0.000000,0.000000,0.000000,0.012500,0.000000,0.120833,0.108333,0.000000,0.325000,0.316667,0.341667,0.004105,0.022846,-0.001063,0.019965,0.031261,0.009394,-0.013198,0.013349,0.022391,-0.111899,0.007173,-0.080292,0,0,0,0,0,-1,-1,-1,-1,-1,0,-1
5,FBDE669A-8B4E-4E6C-A078-C1A71E185C52,IT,Chemicals & Agrochemicals,160,50,0.000000,0.300000,0.000000,0.000000,0.042000,0.000000,0.394000,0.544000,0.084000,0.578000,0.071000,0.608000,0.253729,-0.256560,-0.003851,0.024895,-0.005009,0.010501,0.240312,0.135519,-0.012119,0.376310,0.233629,0.416794,1,0,0,0,0,-1,-1,-1,-1,-1,1,-1
11,93F7AC5D-F331-4033-98D9-AFE7CBB6CE43,CA,Travel & Transportation,69,68,0.847674,0.027273,0.000000,0.012500,0.015909,0.030147,0.108824,0.108824,0.002206,0.158824,0.541176,0.163235,0.077405,-0.051157,-0.001761,0.000471,-0.008144,-0.023016,-0.094700,-0.095355,0.011015,0.024811,0.376389,-0.015519,1,0,0,0,0,-1,-1,-1,-1,-1,1,-1
13,66EBF719-001C-400D-8AEF-AEC9FBCD0AD3,AU,Smart Spaces,116,66,0.866949,0.760577,0.248039,0.950000,0.323276,0.094697,0.368939,0.421212,0.161364,0.743939,0.983333,0.765152,-0.140508,-0.649569,-0.219091,-0.879222,-0.206854,-0.075273,-0.106143,-0.120188,-0.093771,-0.133770,-0.251650,-0.113747,0,0,0,0,0,-1,-1,-1,-1,-1,0,-1


In [6]:
df_display_rank_show.to_excel('results/rec_display_rank_show.xlsx',
                               sheet_name='rank', index=False)

In [7]:
df_display_s_show.to_excel('results/rec_display_full_show.xlsx',
                               sheet_name='full', index=False)

In [ ]:
df_